# Double Trouble Pendulum

The `SwiftyTwoLinks` and `KotlyOtlyDobleDosLinks` apps, or _TwoLinks_ in short, embed a custom motion simulation that solves a systems of dynamic equations for a double pendulum.
This Jupyter Notebook demonstrates how those equations may be determined using the `sympy.physics.mechanics` toolkit.
Each step of the process including importing required modules, defining symbolic constants and variables, defining translational and rotational kinematics, deriving equations of motion, and simulating is included.
Furthermore, the `matplotlib` visualisation module is used to plot states, and create a plot that is used in the application icon.

## Preliminaries

The following modules are imported below, with their summary usage:
 - `sympy` for symbolic algebra
 - `sympy.physics.mechanics` for kinematics and equation of motion derivation
 - `scipy.integrate` for simulating the system of differential equations
 - `numpy` for array manipulation
 - `IPython.display` for rendering equations
 - `matplotlib.pyplot` for plotting

In [ ]:
from sympy import *
from sympy.physics.mechanics import *
from scipy.integrate import solve_ivp
import numpy as np
from IPython.display import display, Latex
import matplotlib.pyplot as plt

The `mechanics_printing()` function is part of `sympy.physics.mechanics`. 
It reduces verbosity of the typeset equation by eliminating $(t)$ from time dependent symbolic terms, and representing time-derivatives by an overdot $\dot{(\, \,)}$ instead of $\partial / \partial t$.
Once it is called as a function, as it is below, these conventions will be followed anytime the `display` function is invoked.

In [ ]:
mechanics_printing()

The function `dtex` is defined below to display LaTeX formatted equations, either taking text-formatted inputs that the user defines, or symbolic equations derived by `sympy`.
It takes a variable number of arguments, and will place an equals sign between them.
Thus its intent is generally that the user may typeset a concise symbol on the left hand side, and the coinciding equation on the right.

In [ ]:
def dtex(*args):
    tex = ''
    for arg in args:
        if isinstance(arg, str):
            # Handle string arguments
            tex += f'${arg}$ $\, = \,$ '
        else:
            # Handle SymPy arguments
            tex += f'${mlatex(arg)}$ $\, = \,$ '
    
    # Trim off the last equals sign
    tex = tex[:-10]  
    display(Latex(tex))    

## Modeling

Below the requisite kinematics and equations of motion are developed symbolically.

### Symbols

For a double pendulum, we define two angles, $\theta_0$ and $\theta_1$, and two angular rates, $\omega_0$ and $\omega_1$, where the index $0$ represents the first link, and the index $1$ represents the second link.
In `sympy.physics.mechanics` we create the time-varying states using the `dynamicsymbols` function, with a list of symbolic terms in string form as the argument.
The vectors $\boldsymbol{q}$ and $\boldsymbol{u}$ represent generalized coordinates and generalized speeds, in radians and radians/sec, respectively, where:

In [ ]:
q = dynamicsymbols('\\theta_0, \\theta_1')
u = dynamicsymbols('\\omega_0, \\omega_1')
dtex('\\boldsymbol{q}', q)
dtex('\\boldsymbol{u}', u)

Initial values are given to these coordinates and speeds to be used in the simulation that follows at the end of the notebook.
In this case, the setup is for the links to be initially held stationary at near vertical angle, then dropped with initially zero angular velocity.

In [ ]:
q0 = np.deg2rad(44), np.deg2rad(44)
u0 = 0.0, 0.0

Several symbolic constants are defined for the link dimensions, mass, gravity, and time.
The $l_k$ symbols denote length, $m_k$ denote mass, and $I_k$ denote moment of inertia about the axis of rotation evaluated at the link center of gravity (CG).
The dimensions $x_0$ and $x_1$ represent distances from the centers of rotation for each link to their respective CG, and the dimension $x_p$ is the distance between the two centers of rotation.
Gravitational acceleration is given by $g_x$ and $g_y$ which are given relative to the fixed inertial frame.
Notably, these values are replaced in _TwoLinks_ by measurements from the device's internal accelerometer.

In [ ]:
l0, l1 = symbols('l_0, l_1')
m0, m1 = symbols('m_0, m_1')
I0, I1 = symbols('I_0, I_1')
x0, x1, xp = symbols('x_0, x_1, x_p')
gx, gy = symbols('g_x, g_y')

Each constant defined above will be substituted with a numeric value prior to the simulation.
The `.subs` method that can be applied to any symbolic equation in `sympy` can take a dictionary as an argument, in which each key in the dictionary will be replaced with its corresponding value.
Note that the keys themselves are the symbols defined above.

In [ ]:
constants = {
    l0: 0.2,
    l1: 0.15,
    m0: 0.1,
    m1: 0.1,
    I0: 5e-6,
    I1: 5e-6,
    x0: 0.1,
    x1: 0.075,
    xp: 0.2,
    gy: -9.8,
    gx: 0.0
}

### Kinematic Equations

`sympy` convention for solvers in the case of systems of equations is to rearrange such that you have vector of equations set equal to zero.
For dynamics system modeling, the implementation of Kane's method requires that such a condition is defined that relates the generalized coordinates to the generalized speeds.
In this case, the generalized speeds, $\omega_k$, are simply the derivative with respect to time of the generalized coordinates, or $\dot{\omega}_k$.
These are referred to as the kinematic equations, which are defined below using a Python list comprehension, with a loop defined over the lists of coordinates and speeds, with the `zip` function used to specify that the two lists will be looped simultaneously.

In [ ]:
kd = [speed - diff(coord) for coord, speed in zip(q, u)]
dtex(Matrix([[0], [0]]), Matrix(kd))

## Frames

The inertial reference frame is defined from which all subsequent frames will be referenced.

In [ ]:
inertial = ReferenceFrame('N')

... Define the inner frame

In [ ]:
innerFrame = inertial.orientnew('A', 'axis', [q[0], inertial.z])
innerFrame.set_ang_vel(inertial, u[0] * inertial.z)

... Define the outer frame

In [ ]:
outerFrame = inertial.orientnew('B', 'axis', [q[1], inertial.z])
outerFrame.set_ang_vel(inertial, u[1] * inertial.z)

display(outerFrame.dcm(inertial))
display(outerFrame.ang_vel_in(inertial))

## Create Points

In [ ]:
origin = Point('O')
origin.set_vel(inertial, Vector(0))

outerPivot = origin.locatenew('P', xp * innerFrame.x)
innerCG = origin.locatenew('0', x0 * innerFrame.x)
outerCG = outerPivot.locatenew('1', x1 * outerFrame.x)
innerEnd = innerCG.locatenew('A', 0.5 * l0 * innerFrame.x)
outerEnd = outerCG.locatenew('B', 0.5 * l1 * outerFrame.x)

outerPivot.v2pt_theory(origin, inertial, innerFrame)
innerCG.v2pt_theory(origin, inertial, innerFrame)
outerCG.v2pt_theory(outerPivot, inertial, outerFrame)

display(innerCG.pos_from(origin))
display(innerCG.vel(inertial))
display(outerCG.pos_from(origin))
display(outerCG.vel(inertial))
display(outerCG.pos_from(origin).to_matrix(inertial))
display(outerCG.acc(inertial))

## Create Bodies

In [ ]:
innerBody = Body('IB', masscenter=innerCG, mass=m0, frame=innerFrame, central_inertia=inertia(innerFrame, 0, 0, I0))
outerBody = Body('OB', masscenter=outerCG, mass=m1, frame=outerFrame, central_inertia=inertia(outerFrame, 0, 0, I1))
bodies = innerBody, outerBody

## Create Gravitational Load

In [ ]:
gvector = gx * inertial.x + gy * inertial.y
loads = [
  (innerBody.masscenter, innerBody.mass * gvector),
  (outerBody.masscenter, outerBody.mass * gvector)
]

## Generate Equations of Motion

In [ ]:
kane = KanesMethod(inertial, q, u, kd)
fr, frstar = kane.kanes_equations(bodies, loads)
dtex('\\boldsymbol{F}_r', simplify(fr))
dtex('\\boldsymbol{F}_R^*', simplify(frstar))

## Get the Simplified Mass Matrix

In [ ]:
M = simplify(kane.mass_matrix_full)
dtex('\\mathbf{M}', M)

## Get the Simplified Forcing Vector

In [ ]:
F = simplify(kane.forcing_full)
dtex('\\mathbf{F}', F)

## Create the Initial Value Problem (IVP)

In [ ]:
massFunction = lambdify(q+u, M.subs(constants))
forcingFunction = lambdify(q+u, F.subs(constants))
def fun(t, x):
  mass = massFunction(*x)
  force = forcingFunction(*x)
  dxdt = np.linalg.inv(mass) @ force
  return dxdt[:, 0].tolist()

## Solve the IVP

In [ ]:
sol = solve_ivp(fun, [0, 1.75], q0+u0, rtol=1e-9)

## Plot Angles and Rates

In [ ]:
plt.subplot(2, 1, 1);
plt.plot(sol.t, sol.y[:2].transpose());
plt.ylabel('Angle [rad]');
plt.legend([f'${angle}$' for angle in q], ncol=2);

plt.subplot(2, 1, 2);
plt.plot(sol.t, sol.y[2:].transpose());
plt.xlabel('Time [sec]')
plt.ylabel('Rate [rad/s]');
plt.legend([f'${rate}$' for rate in u], ncol=2);

## Plot X-Y Position

In [ ]:
innerPosFcn = lambdify(q, innerEnd.pos_from(origin).to_matrix(inertial).transpose().subs(constants).tolist()[0])
outerPosFcn = lambdify(q, outerEnd.pos_from(origin).to_matrix(inertial).transpose().subs(constants).tolist()[0])

innerPos = np.array([innerPosFcn(*q) for q in sol.y[:2].transpose()])
outerPos = np.array([outerPosFcn(*q) for q in sol.y[:2].transpose()])

plt.plot(innerPos[:, 0], innerPos[:, 1]);
plt.scatter(outerPos[:, 0], outerPos[:, 1], 
            s=8*sol.t*3.14, c=sol.t, cmap='rainbow');
plt.axis('off');
plt.xlim([-0.5, 0.5])
plt.tight_layout;

plt.savefig('links.pdf')